## Setup

In [1]:
!python -m spacy download pt

You are using pip version 19.0.3, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/site-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/site-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [2]:
import pandas as pd
import numpy as np

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Load dataset

In [20]:
initial_df = pd.read_csv('../data/CSV/entities/dataset_device_entities_annotated.csv')

## Preprocess

In [21]:
initial_df['original'] = initial_df['original'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
initial_df['anottated'] = initial_df['anottated'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [22]:
import spacy
nlp = spacy.load('pt')

In [23]:
pos_tags = [
    'POS', 'PUNCT', 'SYM','ADJ',
    'NUM', 'DET', 'ADV', 'ADP',
    'X', 'VERB', 'CCONJ', 'NOUN',
    'PROPN', 'PART', 'INTJ', 'SPACE',
    'PRON', 'SCONJ', 'AUX', 'CONJ',
    'START', 'END',
]

labels_list = ['B-DEVICE','I-DEVICE','o']

In [24]:
from sklearn import preprocessing

pos_tags_hotencoder = preprocessing.LabelBinarizer()
pos_tags_hotencoder.fit(pos_tags)

labels_encoder = preprocessing.LabelEncoder()
labels_encoder.fit(labels_list)

LabelEncoder()

In [25]:
def has_digit(string):
    return int(bool(re.match(r'\d', string)))

def is_hour_format(string):
    return int(bool(re.match(r'^[0-9]{1,2}h([0-9]{2,2})?$', string)))

def is_de(string):
    return int(string in ['de', 'da', 'do'])

def generate_features(doc, pos):
    len_doc = len(doc)
    features = []
    token = doc[pos]
    features.append(has_digit(token.text))
    features.append(is_hour_format(token.text))
    features.append(is_de(token.text))
    for i in [-3,-2,-1, 0,1,2,3]:
        feat_pos = pos + i
        if feat_pos < 0:
            features.append('START')
        elif feat_pos >= len_doc:
            features.append('END')
        else:
            features.append(doc[feat_pos].pos_)
    return features

In [26]:
columns = ['f{}'.format(i) for i in range(10)]
columns.append('label')

features_df = pd.DataFrame(columns=columns)

In [27]:
import re

for index, row in initial_df.iterrows():
    doc = nlp(row['original'])
    labels = re.findall('\[@(.*?)\*\]', row['anottated'])
    labels_map = {k: v for k, v in [l.split('#') for l in labels]}  
    for token_index, token in enumerate(doc):
        new_row = generate_features(doc, token_index)
        label = labels_map.get(token.text, 'o')
        new_row.append(label)
        features_df = features_df.append(pd.DataFrame([new_row], columns=columns))
    

In [28]:
features_df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,label
0,0,0,0,START,START,START,VERB,DET,NOUN,ADP,o
0,0,0,0,START,START,VERB,DET,NOUN,ADP,NOUN,o
0,0,0,0,START,VERB,DET,NOUN,ADP,NOUN,PUNCT,B-DEVICE
0,0,0,0,VERB,DET,NOUN,ADP,NOUN,PUNCT,END,o
0,0,0,0,DET,NOUN,ADP,NOUN,PUNCT,END,END,o


In [29]:
print (features_df.groupby('label').size())

features_unique_df = features_df.groupby(columns).size().reset_index(name='count')
features_unique_df = features_unique_df.drop('count', axis=1)

print (features_unique_df.groupby('label').size())

features_df['label_'] = labels_encoder.transform(features_df['label'])
features_unique_df['label_'] = labels_encoder.transform(features_unique_df['label'])

print (features_df.shape)
print (features_unique_df.shape)
print (features_unique_df.head())

label
B-DEVICE     260
I-DEVICE      84
o           1786
dtype: int64
label
B-DEVICE     139
I-DEVICE      42
o           1158
dtype: int64
(2130, 12)
(1339, 12)
   f0  f1  f2   f3   f4     f5    f6    f7     f8   f9 label  label_
0   0   0   0  ADJ  ADJ    ADP  NOUN   END    END  END     o       2
1   0   0   0  ADJ  ADJ  CCONJ   ADJ   END    END  END     o       2
2   0   0   0  ADJ  ADJ   NOUN   DET   NUM  CCONJ    X     o       2
3   0   0   0  ADJ  ADP   NOUN   ADJ   END    END  END     o       2
4   0   0   0  ADJ  ADP   NOUN   ADP  NOUN    END  END     o       2


In [30]:
# only has features for existing categories on dataset
feat_encoded = pd.get_dummies(features_df, columns=features_df.columns[3:-2])
feat_encoded.head(1)

,f0,f1,f2,label,label_,f3_ADJ,f3_ADP,f3_ADV,f3_AUX,f3_CCONJ,...,f9_END,f9_NOUN,f9_NUM,f9_PRON,f9_PROPN,f9_PUNCT,f9_SCONJ,f9_SYM,f9_VERB,f9_X
0,0,0,0,o,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
features_columns = list(set(feat_encoded.columns) - {'label', 'label_'})

print (features_columns)

['f7_SYM', 'f5_PRON', 'f7_ADJ', 'f8_SYM', 'f6_PROPN', 'f3_START', 'f4_START', 'f4_NUM', 'f8_ADV', 'f6_NOUN', 'f3_PUNCT', 'f8_CCONJ', 'f8_PRON', 'f5_ADV', 'f8_PROPN', 'f8_PUNCT', 'f9_ADV', 'f6_DET', 'f6_SCONJ', 'f3_SYM', 'f9_PROPN', 'f4_PROPN', 'f6_NUM', 'f5_CCONJ', 'f5_ADJ', 'f4_NOUN', 'f4_PRON', 'f3_CCONJ', 'f4_DET', 'f7_PRON', 'f4_CCONJ', 'f6_SYM', 'f4_ADP', 'f3_ADJ', 'f3_NUM', 'f9_DET', 'f7_AUX', 'f5_SYM', 'f2', 'f9_ADJ', 'f9_ADP', 'f7_NUM', 'f3_SCONJ', 'f6_ADV', 'f5_DET', 'f6_CCONJ', 'f5_NUM', 'f6_PUNCT', 'f4_VERB', 'f3_ADV', 'f4_PUNCT', 'f9_X', 'f8_VERB', 'f4_AUX', 'f3_PRON', 'f3_NOUN', 'f8_ADP', 'f9_SCONJ', 'f5_ADP', 'f9_CCONJ', 'f3_AUX', 'f3_ADP', 'f9_NOUN', 'f7_X', 'f4_SCONJ', 'f5_PROPN', 'f3_DET', 'f6_VERB', 'f5_PUNCT', 'f5_NOUN', 'f8_NOUN', 'f6_AUX', 'f7_ADP', 'f7_PROPN', 'f7_END', 'f9_VERB', 'f7_SCONJ', 'f6_ADJ', 'f7_PUNCT', 'f9_PRON', 'f5_START', 'f3_VERB', 'f6_ADP', 'f9_NUM', 'f7_NOUN', 'f5_SCONJ', 'f3_PROPN', 'f8_END', 'f7_CCONJ', 'f0', 'f4_ADJ', 'f6_PRON', 'f1', 'f9_SYM'

In [32]:
len(features_columns)

110

In [33]:
from sklearn.model_selection import train_test_split

trainData, testData, trainLabels, testLabels = train_test_split(feat_encoded[features_columns], feat_encoded['label_'], test_size=0.2, random_state=42)
testLabels = testLabels.astype(int)
trainLabels = trainLabels.astype(int)
len(trainData), len(testData), len(trainLabels), len(testLabels)

(1704, 426, 1704, 426)

In [34]:
len(trainData.columns), len(testData.columns)

(110, 110)

## Classifier

In [36]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

models = [
    ('GradientBoostingClassifier', GradientBoostingClassifier(), {}),
    ('LinearSVC', LinearSVC(), {}),
    ('SGD', SGDClassifier(), {}),
]

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import strip_accents_ascii
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV 


results = []
names = []
best_model = None
max_score = -1

for name, model, cls_params in models:
    pipe = Pipeline([('clf', model)])
  
    gs_cls = GridSearchCV(pipe, cls_params, n_jobs=1)
    gs_cls = gs_cls.fit(trainData, trainLabels)
  
    print ("Model: "+ name)
    print ("Best cross-validation score: " + str(gs_cls.best_score_))
    print ("Best parameters: " + str(gs_cls.best_params_))

    predictions = gs_cls.predict(testData)
    score = gs_cls.score(testData, testLabels)
    print ("Accuracy: " + str(score))

    if score > max_score:
        max_score = score
        best_model = gs_cls

    cm = confusion_matrix(testLabels, predictions)
    for i, l in enumerate(cm):
        print ('{0: <10} {1}'.format(labels_list[i], ''.join(['{0: <3}'.format(x) for x in l])))
    
    print (classification_report(testLabels, predictions))
                       
    names.append(name)
    results.append(score)

Model: GradientBoostingClassifier
Best cross-validation score: 0.9383802816901409
Best parameters: {}
Accuracy: 0.9624413145539906
B-DEVICE   44 1  7  
I-DEVICE   0  10 3  
o          5  0  356
             precision    recall  f1-score   support

          0       0.90      0.85      0.87        52
          1       0.91      0.77      0.83        13
          2       0.97      0.99      0.98       361

avg / total       0.96      0.96      0.96       426

Model: LinearSVC
Best cross-validation score: 0.9395539906103286
Best parameters: {}
Accuracy: 0.960093896713615
B-DEVICE   46 1  5  
I-DEVICE   0  10 3  
o          7  1  353
             precision    recall  f1-score   support

          0       0.87      0.88      0.88        52
          1       0.83      0.77      0.80        13
          2       0.98      0.98      0.98       361

avg / total       0.96      0.96      0.96       426

Model: SGD
Best cross-validation score: 0.9307511737089202
Best parameters: {}
Accuracy: 0.948

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.6/site-packages/s

In [38]:
import matplotlib
from matplotlib import pyplot as plt
import numpy as np

fig, ax = plt.subplots(1,1, figsize=(16, 6))
fig.suptitle('Algorithims Accuracy Comparison')
plt.bar(names, results, )
plt.ylim(0,1)
plt.yticks(np.arange(0,1.1,0.1))
for i, v in enumerate(names):
    ax.text(v, results[i] + 0.05, '%.4f' %(results[i]), ha='center', va='center')

    
plt.show()

<Figure size 1600x600 with 1 Axes>

## Save model

In [39]:
import pickle

filename = '../api/models/entities_model.sav' 
with open(filename, 'wb') as f:
    pickle.dump(best_model, f, protocol=2)
    
filename = '../api/models/entities_label_binarizer.sav' 
with open(filename, 'wb') as f:
    pickle.dump(labels_encoder, f, protocol=2)

## Predict

In [40]:
def process(sent):
    d = pd.DataFrame(columns=columns[:-1])
    doc = nlp(sent)
    for i, token in enumerate(doc):
        new_row = generate_features(doc, i)
        d = d.append(pd.DataFrame([new_row], columns=columns[:-1]))
    d = pd.get_dummies(d)
    
    # Get missing columns in the training test
    missing_cols = set( features_columns ) - set( d.columns )
    
    # Add a missing column in test set with default value equal to 0
    for c in missing_cols:
        d[c] = 0
        
    # Ensure the order of column in the test set is in the same order than in train set
    return d[features_columns]

def predict(sent):
    d = process(sent)
    p = best_model.predict(d)
    return labels_encoder.inverse_transform(p)


In [41]:
for s in initial_df['original'][:10]:
    print (s, predict(s))

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Liga a televisao por favor. ['o' 'o' 'B-DEVICE' 'o' 'o' 'o']
Ligue o ar condicionado. ['o' 'o' 'B-DEVICE' 'I-DEVICE' 'o']
Ar condicionado ligado. ['B-DEVICE' 'I-DEVICE' 'o' 'o']


/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Liga a luz do quarto ai. ['o' 'o' 'B-DEVICE' 'o' 'o' 'o' 'o']
Liga meu computador pra mim. ['o' 'o' 'B-DEVICE' 'o' 'o' 'o']
Acende a luz da cozinha por favor. ['o' 'o' 'B-DEVICE' 'o' 'o' 'o' 'o' 'o']


/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Liga o som. ['o' 'o' 'B-DEVICE' 'o']
Bota um som ai pra mim. ['o' 'o' 'B-DEVICE' 'o' 'o' 'o' 'o']
Bota uma musica ai. ['o' 'o' 'B-DEVICE' 'o' 'o']
Meu quarto esta um pouco escuro. ['o' 'o' 'o' 'o' 'o' 'o' 'o']


/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [42]:
predict('Liga o ar condicionado todo dia a noite')

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['o', 'o', 'B-DEVICE', 'o', 'o', 'B-DEVICE', 'o', 'o'], dtype='<U8')

In [43]:
predict('liga o ar condicionado')

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['o', 'o', 'B-DEVICE', 'I-DEVICE'], dtype='<U8')

In [44]:
predict('acende a luz do meu quarto')

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['o', 'o', 'B-DEVICE', 'o', 'o', 'o'], dtype='<U8')

In [45]:
predict('liga todas as luzes das salas de reunião')

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['o', 'o', 'o', 'B-DEVICE', 'o', 'o', 'o', 'o'], dtype='<U8')

In [46]:
predict('liga a televisao as 5 horas')

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['o', 'o', 'o', 'o', 'o', 'o'], dtype='<U8')